In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/Aditya Kyatham/Documents/BE FINAL/nsl-kdd/NSL_KDD-master/Small Training Set_5class.csv", index_col=None)

In [3]:
df=df.drop(['land','su_attempted','num_outbound_cmds','is_host_login','urgent','num_failed_logins','su_attempted','num_file_creations','num_shells','srv_diff_host_rate'], axis = 1) 


In [4]:
df.head()

,duration,"""protocol_type""","""service""",flag,"""src_bytes""","""dst_bytes""",wrong_fragment,"""hot""",logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_diff_srv_rate.1,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,dos
3,0,tcp,http,SF,232,8153,0,0,1,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,1,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [5]:
y = df["label"].values
from collections import Counter
#num = Counter(y)
#n=num.get('normal')
Counter(y)

Counter({'normal': 516, 'dos': 380, 'r2l': 13, 'probe': 91, 'u2r': 11})

In [6]:
from sklearn.preprocessing import LabelEncoder
encodings = dict()
for c in df.columns:
    #print df[c].dtype
    if df[c].dtype == "object":
        encodings[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings[c]
        df[c] = encodings[c].fit_transform(df[c])

In [7]:
y = df.pop("label").values
X = df.values

In [8]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()


C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [9]:
from sklearn.preprocessing import StandardScaler #normalization
X = StandardScaler().fit_transform(X)

In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
principalDfX = pd.DataFrame(data = principalComponents)

In [11]:
principalDfX.head()
print(Counter(y))

Counter({1: 516, 0: 380, 2: 91, 3: 13, 4: 11})


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(principalDfX, y, test_size=0.2, random_state=42)#random for splitting same data when run again.
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [13]:
from sklearn.svm import SVC

clf = SVC(gamma='auto',decision_function_shape='ovo')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [14]:
y_pred=clf.predict(X_test)
print(Counter(y_pred))
print(Counter(y_test))

Counter({1: 117, 0: 63, 2: 23})
Counter({1: 108, 0: 64, 2: 26, 4: 3, 3: 2})


In [15]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
 
results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test, y_pred))
print('Report : ')
print(classification_report(y_test, y_pred))


Confusion Matrix :
[[ 61   3   0   0   0]
 [  0 107   1   0   0]
 [  2   2  22   0   0]
 [  0   2   0   0   0]
 [  0   3   0   0   0]]
Accuracy Score : 0.9359605911330049
Report : 
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        64
           1       0.91      0.99      0.95       108
           2       0.96      0.85      0.90        26
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         3

    accuracy                           0.94       203
   macro avg       0.57      0.56      0.56       203
weighted avg       0.91      0.94      0.92       203



C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
predicted = cross_val_predict(clf, X_train, y_train, cv=skf)
print('Accuracy Score :',accuracy_score(y_train, predicted))
print('Report : ')
print(classification_report(y_train, predicted))


C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Accuracy Score : 0.9195544554455446
Report : 
              precision    recall  f1-score   support

           0       0.97      0.90      0.93       316
           1       0.88      1.00      0.94       408
           2       0.96      0.80      0.87        65
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00         8

    accuracy                           0.92       808
   macro avg       0.56      0.54      0.55       808
weighted avg       0.90      0.92      0.91       808



C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
